In [0]:
import numpy as np
import gzip
import pickle as pkl
import matplotlib.pyplot as plt
import datetime
import sys
import torch

In [0]:
torch.__version__

'1.1.0'

In [0]:
!nvidia-smi

Mon May 13 19:48:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    18W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Thats a good gpu, with no pesky graphics taking up storage either

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd /gdrive/My\ Drive/cs445/

/gdrive/My Drive/cs445


In [0]:
import neuralnetworks_pytorch as nn

In [0]:
ls

final_training_decade.ipynb  neuralnetworks_pytorch.py  pickles/  __pycache__/


In [0]:
#Xbatch1 = pkl.load(open("./pickles/training_batches/batch0.pkl", "rb"))
#Tbatch1 = pkl.load(open("./pickles/training_batches/batch0_targets.pkl", "rb"))
#Xbatch2 = pkl.load(open("./pickles/training_batches/batch1.pkl", "rb"))
#Tbatch2 = pkl.load(open("./pickles/training_batches/batch1_targets.pkl", "rb"))
#Xbatch3 = pkl.load(open("./pickles/training_batches/batch2.pkl", "rb"))
#Tbatch3 = pkl.load(open("./pickles/training_batches/batch2_targets.pkl", "rb"))
#Xbatch1_test = pkl.load(open("./pickles/test_batches/batch0.pkl", "rb"))
#Tbatch1_test = pkl.load(open("./pickles/test_batches/batch0_targets.pkl", "rb"))

In [0]:
Xbatch = pkl.load(open("./pickles/train_batches1-5/batch0.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches1-5/batch0_targets.pkl", "rb"))

In [0]:
Xbatch_test = pkl.load(open("./pickles/test_batches1-5/batch0.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches1-5/batch0_targets.pkl", "rb")) 

In [0]:
Xbatch.shape, Xbatch_test.shape

((6515, 55, 110, 3), (1629, 55, 110, 3))

In [0]:
Xbatch.nbytes #118 MB, used to be 450 MB a batch before downsampling

118247250

In [0]:
makes = np.unique(Tbatch[:, 0])
styles = np.unique(Tbatch[:,1])
years = np.unique(Tbatch[:,2])
years, makes, styles

(array(['1991', '1993', '1994', '1997', '1998', '1999', '2000', '2001',
        '2002', '2006', '2007', '2008', '2009', '2010', '2011', '2012'],
       dtype='<U54'),
 array(['AM', 'Acura', 'Aston', 'Audi', 'BMW', 'Bentley', 'Bugatti',
        'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Daewoo', 'Dodge',
        'Eagle', 'FIAT', 'Ferrari', 'Fisker', 'Ford', 'GMC', 'Geo',
        'HUMMER', 'Honda', 'Hyundai', 'Infiniti', 'Isuzu', 'Jaguar',
        'Jeep', 'Lamborghini', 'Land', 'Lincoln', 'MINI', 'Maybach',
        'Mazda', 'McLaren', 'Mercedes-Benz', 'Mitsubishi', 'Nissan',
        'Plymouth', 'Porsche', 'Ram', 'Rolls-Royce', 'Scion', 'Spyker',
        'Suzuki', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo', 'smart'],
       dtype='<U54'),
 array(['Convertible', 'Coupe', 'Hatchback', 'Minivan', 'SUV', 'Sedan',
        'Truck', 'Van', 'Wagon'], dtype='<U54'))

Make target classes for decade of produciton

In [0]:
for target in Tbatch:
    if int(target[2]) < 2000:
        target[2] = 0
    elif 2000 <= int(target[2]) < 2010:
        target[2] = 1
    else:
        target[2] = 2

In [0]:
for target in Tbatch_test:
    if int(target[2]) < 2000:
        target[2] = 0
    elif 2000 <= int(target[2]) < 2010:
        target[2] = 1
    else:
        target[2] = 2

In [0]:
n_outputs = 3

In [0]:
Ttrain = Tbatch[:, 2]
Ttest = Tbatch_test[:, 2]

Change to the channels first format, which is what is expected by pytorch

In [0]:
Xbatch = np.rollaxis(Xbatch, 3, 1)
Xbatch_test = np.rollaxis(Xbatch_test, 3, 1)

In [0]:
input_size_in_pixels = 3

Image is 55 by 110. For convolution size, we can start with 7x7, then 5x5 down to 3x3

In [0]:
n_hiddens_by_layer = [300, 400, 500, 800, 5000, 10000]
batch_size = 200
learning_rate = .00001 #.00001 great with 16
n_iterations = 6
n_conv_layers = 5

window = [7, 5, 3, 3]
stride = [2, 2, 2, 2]
relu = True
gpu = True

In [0]:
input_height = Xbatch.shape[2]
input_width = Xbatch.shape[3]

In [0]:
nnet = nn.NeuralNetworkClassifier_Pytorch(input_size_in_pixels, 
                                          n_hiddens_by_layer, 
                                          n_outputs, 
                                          relu,gpu, 
                                          n_conv_layers, window, stride, input_height, input_width)

NeuralNetworkClassifier_Pytorch created on cuda


In [0]:
nnet.train(Xbatch, Ttrain.astype(np.uint8), Xbatch_test, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 0.9168, acc = 66.18
Iteration 2, cost = 0.7342, acc = 65.87
Iteration 3, cost = 0.6833, acc = 66.05
Iteration 4, cost = 0.6269, acc = 66.24
Iteration 5, cost = 0.5655, acc = 65.87
Iteration 6, cost = 0.5040, acc = 66.73


In [0]:
def percent_correct(actual, predicted):
    return 100 * np.mean(actual == predicted)

In [0]:
classes, _, _ = nnet.use(Xbatch_test)
percent = percent_correct(Ttest.astype(np.uint8), classes)
percent


66.7280540208717

We need to use this on a CPU. Let's try and get it back from the GPU so we can use it on the server

In [0]:
cpu_nnet = nnet.to(torch.device('cpu'))
cpu_nnet.device = 'cpu'

In [0]:
pkl.dump(cpu_nnet, open('decade_nnet', 'wb'))

In [0]:
cpu_nnet.device 

'cpu'